In [1]:
'''数据清洗'''
import pandas as pd
import os

DATA_PRE_DIR = './datasets/radar'
DATA_AFTER_DIR = './datasets/cache'

FEATURES_COLUMNS = ['航班号(I170)', '经纬度(I130)', '系统接收时间', '地速(I160)', '几何高度(I140)', '航向(I160)']
FEATURES_COLUMNS = ['航班号(I170)', '高精度经纬度(I131)', '高精度位置报接收时间(I073&I074)', ]


def get_longitude(value: str, **extra):
    if value is None or value is pd.NA or value.strip() == '':
        return pd.NA
    return float(value.split(',')[0])


def get_latitude(value: str, **extra):
    if value is None or value is pd.NA or value.strip() == '':
        return pd.NA
    return float(value.split(',')[1])

In [39]:
p = os.path.join(DATA_PRE_DIR, '20210401.txt')
df: pd.DataFrame = pd.read_table(p, sep='\t', encoding='UTF-8')
# 空串替换为NA
df.replace(to_replace=r'^\s*$', value=pd.NA, regex=True, inplace=True)
# 0.00替换为NA
df.replace(to_replace=r'^\s*?0*?\.?0*?\s*?$', value=pd.NA, regex=True, inplace=True)
# 无航班号和时间的删除
df.dropna(axis=0, how='any', subset=['航班号(I170)', '系统接收时间'], inplace=True)
df = df.loc[:, FEATURES_COLUMNS].copy(deep=True)
# 重命名
df.rename(columns={
    '系统接收时间': '时间',
    '航班号(I170)': '航班号',
    '地速(I160)': '速度',
    '几何高度(I140)': '高度',
    '航向(I160)': '航向'
}, inplace=True)
# 经纬度提取
df['经度'] = df.loc[:, '经纬度(I130)'].apply(get_longitude)
df['纬度'] = df.loc[:, '经纬度(I130)'].apply(get_latitude)
df.drop(['经纬度(I130)'], axis=1, inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: './datasets/cache\\20210401.txt'

In [18]:
pnames = set(df['航班号'].to_list())
dt = df.loc[df['航班号'] == 'TEST002 ']
dt

,航班号,时间,速度,高度,航向,经度,纬度
128,TEST002,2020-03-31 16:00:28.872,<NA>,<NA>,<NA>,<NA>,<NA>
129,TEST002,2020-03-31 16:00:28.872,<NA>,<NA>,<NA>,<NA>,<NA>
130,TEST002,2020-03-31 16:00:28.872,<NA>,<NA>,<NA>,<NA>,<NA>
131,TEST002,2020-03-31 16:00:28.873,<NA>,<NA>,<NA>,<NA>,<NA>
665,TEST002,2020-03-31 16:00:30.874,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...
1564248,TEST002,2020-03-31 19:53:35.408,<NA>,<NA>,<NA>,<NA>,<NA>
1564333,TEST002,2020-03-31 19:53:36.413,<NA>,<NA>,<NA>,<NA>,<NA>
1564334,TEST002,2020-03-31 19:53:36.413,<NA>,<NA>,<NA>,<NA>,<NA>
1564335,TEST002,2020-03-31 19:53:36.413,<NA>,<NA>,<NA>,<NA>,<NA>


In [4]:
df_bak = df.copy(deep=True)

In [38]:
df.loc[1564335:1564340, ('航班号', '纬度', '经度')]

,航班号,纬度,经度


In [ ]:
df.interpolate(method='time', axis=0, limit=2, inplace=True)

In [ ]:
df.loc[1564335:1564340, ('航班号', '经度')]

In [20]:
dt = df.loc[df['航班号'] == 'TEST002 ']

In [25]:
all(dt['速度'].isna())

True

In [28]:
'''数据清洗'''
import pandas as pd
import os
from datetime import datetime

DATA_PRE_DIR = './datasets/cache'
DATA_AFTER_DIR = './datasets/train'

FEATURES_COLUMNS = ['航班号', '时间', '经度', '纬度', '速度', '高度', '航向']


for p in os.listdir(DATA_PRE_DIR):
    p = os.path.join(DATA_PRE_DIR, p)
    # TODO 归一化
    df = pd.read_csv(p, sep=',')
    break

In [35]:
def convert_datetime_numric(value: datetime, **extra):
    return value.timestamp()

df['时间'] = df.loc[:, '时间'].apply(pd.to_datetime, errors='raise', format='%Y-%m-%d %H:%M:%S.%f')
df['时间'] = df.loc[:, '时间'].apply(convert_datetime_numric)

In [36]:
df

,时间,航班号,经度,纬度,速度,高度,航向
0,1.585674e+09,AAR9751,112.957840,38.014561,726.38,7193.28,244.48
1,1.585674e+09,AAR9751,112.957840,38.014561,726.38,7193.28,244.48
2,1.585674e+09,AAR9751,112.953896,38.013103,726.38,7193.28,244.48
3,1.585674e+09,AAR9751,112.953896,38.013103,726.38,7193.28,244.48
4,1.585674e+09,AAR9751,112.949966,38.011600,726.38,7193.28,244.48
...,...,...,...,...,...,...,...
38166,1.585678e+09,AAR9751,108.765274,34.431236,0.41,449.58,5.62
38167,1.585678e+09,AAR9751,108.765274,34.431236,0.41,449.58,5.62
38168,1.585678e+09,AAR9751,108.765274,34.431236,0.41,449.58,5.62
38169,1.585678e+09,AAR9751,108.765274,34.431236,0.41,449.58,5.62
